In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np 
import pandas as pd 
import os 
import kagglehub
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
print(tf.__version__)

2025-03-08 04:58:46.936166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 04:58:46.961123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-08 04:58:46.980809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-08 04:58:46.986755: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 04:58:47.001479: I tensorflow/core/platform/cpu_feature_guar

2.17.0


In [2]:
dataset_path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-small")
print("Path to dataset files:", dataset_path)

Path to dataset files: /nfs/home/mjc3869/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-small/versions/1


In [3]:
dataset_path = '/nfs/home/mjc3869/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-small/versions/1'

In [4]:
# Define dataset paths
image_folder = os.path.join(dataset_path, "images")
metadata_path = os.path.join(dataset_path, "styles.csv")

# Step 2: Load metadata
df = pd.read_csv(metadata_path, on_bad_lines='skip')

# Filter dataset (only use images present in folder)
df["image_path"] = df["id"].astype(str) + ".jpg"
df = df[df["image_path"].isin(os.listdir(image_folder))]

# Select a subset of classes for simplicity
df = df[df['masterCategory'].isin(['Apparel', 'Footwear', 'Accessories'])]

# Encode labels
label_map = {label: idx for idx, label in enumerate(df["masterCategory"].unique())}
df["category_id"] = df["masterCategory"].map(label_map)

# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["category_id"], random_state=42)

In [5]:
from keras_preprocessing.image import ImageDataGenerator

image_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col="image_path", y_col="masterCategory",
    target_size=image_size, batch_size=batch_size, class_mode="categorical", subset="training"
)

val_generator = datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col="image_path", y_col="masterCategory",
    target_size=image_size, batch_size=batch_size, class_mode="categorical", subset="validation"
)

Found 26807 validated image filenames belonging to 3 classes.
Found 6701 validated image filenames belonging to 3 classes.


In [6]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_map), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

2025-03-08 05:00:11.040842: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 654 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2025-03-08 05:00:11.042814: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9461 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2025-03-08 05:00:11.044522: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9461 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3d:00.0, compute capability: 7.5
2025-03-08 05:00:11.046456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 2531 MB memory:  -> device: 3, name: NVIDIA GeForce RTX 

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


/nfs/home/mjc3869/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


I0000 00:00:1741410021.356413 3807047 service.cc:146] XLA service 0x7f9fac0029b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741410021.356472 3807047 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741410021.356480 3807047 service.cc:154]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741410021.356484 3807047 service.cc:154]   StreamExecutor device (2): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741410021.356488 3807047 service.cc:154]   StreamExecutor device (3): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741410021.356492 3807047 service.cc:154]   StreamExecutor device (4): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741410021.356496 3807047 service.cc:154]   StreamExecutor device (5): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741410021.3564

  3/838 ━━━━━━━━━━━━━━━━━━━━ 43s 52ms/step - accuracy: 0.3733 - loss: 1.1554   

I0000 00:00:1741410025.651773 3807047 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


838/838 ━━━━━━━━━━━━━━━━━━━━ 119s 130ms/step - accuracy: 0.7904 - loss: 0.5100 - val_accuracy: 0.8966 - val_loss: 0.2506
Epoch 2/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 114s 136ms/step - accuracy: 0.9117 - loss: 0.2262 - val_accuracy: 0.8614 - val_loss: 0.3273
Epoch 3/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 112s 134ms/step - accuracy: 0.9271 - loss: 0.1929 - val_accuracy: 0.9346 - val_loss: 0.1741
Epoch 4/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 101s 120ms/step - accuracy: 0.9303 - loss: 0.1808 - val_accuracy: 0.9306 - val_loss: 0.1775
Epoch 5/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 101s 120ms/step - accuracy: 0.9362 - loss: 0.1733 - val_accuracy: 0.9419 - val_loss: 0.1647
Epoch 6/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 103s 123ms/step - accuracy: 0.9396 - loss: 0.1648 - val_accuracy: 0.9479 - val_loss: 0.1517
Epoch 7/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 102s 121ms/step - accuracy: 0.9411 - loss: 0.1586 - val_accuracy: 0.9460 - val_loss: 0.1559
Epoch 8/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 103s 123ms/step - accuracy: 0.9444 - loss: 0.15

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col="image_path", y_col="masterCategory",
    target_size=image_size, batch_size=batch_size, class_mode="categorical", shuffle=False
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.2f}")

# Save Model
model.save("fashion_classifier_resnet.h5")

Found 8377 validated image filenames belonging to 3 classes.


/nfs/home/mjc3869/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


262/262 ━━━━━━━━━━━━━━━━━━━━ 30s 114ms/step - accuracy: 0.9592 - loss: 0.1260


Test Accuracy: 0.96
